In [69]:
import os
import json
import dotenv
from openai import AzureOpenAI
import gradio as gr
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import Chroma
import time
from langchain.embeddings import AzureOpenAIEmbeddings
from chromadb.config import Settings
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser

In [70]:
dotenv.load_dotenv()

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_APIKEY"),  
    api_version=os.getenv('OPEN_API_VERSION'),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [71]:
loader = CSVLoader("./philosophy.csv")
pages = loader.load_and_split()

In [72]:
deployment_name = "text-embedding-01"

# Define chunk size and pause duration
chunk_size = 150
pause_duration = 1  # pause for 5 seconds


In [73]:
chunk = pages[0:chunk_size]
embeddings = AzureOpenAIEmbeddings(azure_deployment=deployment_name)
vectorstore = Chroma.from_documents(
    documents=chunk,
    embedding=embeddings,
    client_settings=Settings(),
    persist_directory="philosophy_head.db"
)

In [74]:
# Create Chroma vectorstore from documents in chunks
for i in range(chunk_size, len(pages), chunk_size):
    print(f"Processing documents {i} to {i+chunk_size}")
    chunk = pages[i:i+chunk_size]
    vectorstore.aadd_documents(
        documents=chunk,
        embedding=embeddings,
        client_settings=Settings()
    )
    time.sleep(pause_duration)

retriever = vectorstore.as_retriever()

Processing documents 150 to 300


/var/folders/w5/9wvkhys54sd_f1x0tgfg9rkm0000gn/T/ipykernel_74203/3273429229.py:5: RuntimeWarning: coroutine 'VectorStore.aadd_documents' was never awaited
  vectorstore.aadd_documents(


Processing documents 300 to 450
Processing documents 450 to 600
Processing documents 600 to 750
Processing documents 750 to 900
Processing documents 900 to 1050
Processing documents 1050 to 1200
Processing documents 1200 to 1350
Processing documents 1350 to 1500
Processing documents 1500 to 1650
Processing documents 1650 to 1800
Processing documents 1800 to 1950
Processing documents 1950 to 2100
Processing documents 2100 to 2250
Processing documents 2250 to 2400
Processing documents 2400 to 2550
Processing documents 2550 to 2700
Processing documents 2700 to 2850
Processing documents 2850 to 3000
Processing documents 3000 to 3150
Processing documents 3150 to 3300
Processing documents 3300 to 3450
Processing documents 3450 to 3600
Processing documents 3600 to 3750
Processing documents 3750 to 3900
Processing documents 3900 to 4050
Processing documents 4050 to 4200
Processing documents 4200 to 4350
Processing documents 4350 to 4500
Processing documents 4500 to 4650
Processing documents 46

In [75]:
temperature = 0.2
deployment_name = "gpt-model-01"
llm = AzureChatOpenAI(temperature=temperature, azure_deployment=deployment_name)

def format_docs(docs):
    doc = "\n\n".join(doc.page_content for doc in docs)
    return doc

In [76]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

condense_q_system_prompt = """Given a chat history and the latest user question \
which might reference the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
condense_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
condense_q_chain = condense_q_prompt | llm | StrOutputParser()

In [77]:
from langchain.schema.messages import AIMessage, HumanMessage

condense_q_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large language model"),
        ],
        "question": "What is meant by large",
    }
)

'What is the definition of "large" in the context of a language model?'

In [78]:
qa_system_prompt = """You are an assistant for question-answering tasks who is an expert in philosophy. Answer the question only if it is related to philosophy and otherwise say that it isn't part of your domain. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def condense_question(input: dict):
    if input.get("chat_history"):
        return condense_q_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(context=condense_question | retriever | format_docs)
    | qa_prompt
    | llm
)

In [79]:
answer = rag_chain.invoke(
    {
        "chat_history": [
            HumanMessage(content="What does LLM stand for?"),
            AIMessage(content="Large language model"),
        ],
        "question": "What is meant by large",
    }
)

In [83]:
import azure.cognitiveservices.speech as speechsdk

def tts(text):
    # This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
    speech_config = speechsdk.SpeechConfig(subscription=os.environ.get('SPEECH_KEY'), region=os.environ.get('SPEECH_REGION'))
    audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)

    # The language of the voice that speaks.
    speech_config.speech_synthesis_voice_name='en-US-JennyNeural'

    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

    # Get text from the console and synthesize to the default speaker.
    print("Enter some text that you want to speak >")
    text = input()

    speech_synthesis_result = speech_synthesizer.speak_text_async(text).get()

    if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print("Speech synthesized for text [{}]".format(text))
    elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_synthesis_result.cancellation_details
        print("Speech synthesis canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            if cancellation_details.error_details:
                print("Error details: {}".format(cancellation_details.error_details))
                print("Did you set the speech resource key and region values?")

In [80]:
def produce_response(question, history):
    """
      message: new message from the user
      history: history of messages from user
    """
    messages = []
    for e in history:
      if e[0] == None:
        messages.append(AIMessage(content=""))
      else:
        messages.append(HumanMessage(content=e[0]))
      messages.append(AIMessage(content=e[1]))

    messages=messages[:10]
    answer = rag_chain.invoke(
        {
          "chat_history": messages,
          "question": question,
        }
    ).content

    tts(answer)

    return answer

In [81]:
import azure.cognitiveservices.speech as speechsdk

def recognize_from_microphone():
    # This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
    speech_config = speechsdk.SpeechConfig(subscription=os.environ.get('SPEECH_KEY'), region=os.environ.get('SPEECH_REGION'))
    speech_config.speech_recognition_language="en-US"

    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print("Speak into your microphone.")
    speech_recognition_result = speech_recognizer.recognize_once_async().get()

    if speech_recognition_result.reason == speechsdk.ResultReason.RecognizedSpeech:
        return speech_recognition_result.text
    elif speech_recognition_result.reason == speechsdk.ResultReason.NoMatch:
        return f'No speech could be recognized: {speech_recognition_result.no_match_details}'
    elif speech_recognition_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_recognition_result.cancellation_details
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")
            return f'Speech Recognition canceled: {cancellation_details.error_details}'



In [82]:
# def process_input(text_input, speech_input):
    if speech_input:
        text_input = recognize_from_microphone()
    response = produce_response(text_input)
    return response

# demo = gr.Interface(
#     process_input,
#     ['textbox', 'textbox'],
#     'textbox',
#     title="OpenAI Chatbot Example",
#     description="A chatbot example for QCRI Generative AI Hackathon 2023",
# )

# demo.launch()

demo = gr.ChatInterface(
  produc  response,
#   titl  "OpenAI Chatb# ot Example",
  de  ription="A chatbot example for#  QCRI Generative AI Hackathon 2023",
    atb# ot=gr.Chatbot(value=[(None, "Welcome")])
  )

# if __name__ == "__main__":
demo.launch()

IndentationError: unexpected indent (2853146152.py, line 2)